## Задание
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД
## Задание *
Написать скрипт, который скачивает фотографии всех строящихся объектов с сайта "наш.дом.рф"

In [1]:
# import all required libs
import requests
import time
from datetime import datetime
from pprint import pprint
import pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import sqlite3


In [2]:
%%time
# Делаем запрос через API
# Проходим циклом пока в списке сайта есть объекты 
# Формируем список с объектами и далее по нему проходим циклом и записываем информацию по объектам в buildings


session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

buildings = []
offset = 0
limit = 1000
length = 1


while length > 0:
    start = datetime.now()
    object_list = []
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset}&limit={limit}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
    res = session.get(url)
    res_json = res.json()
    for i in res_json['data']['list']:
        object_list.append(i['objId'])
    offset += limit
    length = len(res_json['data']['list'])
    for i in object_list:
        url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{i}'
        res = session.get(url)
        res_json = res.json()
        buildings.append(res_json)
    finish = datetime.now()
    print(f'Кол-во выгруженных домов в датасет - {len(buildings)}', f'время выполнения прохода - {finish - start}')
    

Кол-во выгруженных домов в датасет - 1000 время выполнения прохода - 0:01:46.733117
Кол-во выгруженных домов в датасет - 2000 время выполнения прохода - 0:01:49.085027
Кол-во выгруженных домов в датасет - 3000 время выполнения прохода - 0:01:33.647245
Кол-во выгруженных домов в датасет - 4000 время выполнения прохода - 0:01:29.215200
Кол-во выгруженных домов в датасет - 5000 время выполнения прохода - 0:01:34.490351
Кол-во выгруженных домов в датасет - 6000 время выполнения прохода - 0:01:29.021178
Кол-во выгруженных домов в датасет - 7000 время выполнения прохода - 0:01:44.628539
Кол-во выгруженных домов в датасет - 8000 время выполнения прохода - 0:01:31.522136
Кол-во выгруженных домов в датасет - 9000 время выполнения прохода - 0:01:33.677934
Кол-во выгруженных домов в датасет - 10000 время выполнения прохода - 0:01:39.016033
Кол-во выгруженных домов в датасет - 10535 время выполнения прохода - 0:00:51.576663
Кол-во выгруженных домов в датасет - 10535 время выполнения прохода - 0:00

In [3]:
# Выведем количество объектов недвижимости, которое у нас получилось в итоге
len(buildings)

10535

In [4]:
# Формируем датафрейм с помощью json_normalize и выведем 5 случайных строк
df = pd.json_normalize(buildings)
df.sample(5)

,errcode,data.id,data.pdId,data.developer.devId,data.developer.devShortCleanNm,data.developer.devShortNm,data.developer.devFullCleanNm,data.developer.problObjCnt,data.developer.buildObjCnt,data.developer.comissObjCnt,...,data.developer.bankruptStage.bankruptStageDesc,data.developer.bankruptStage.bankruptLawUrl,data.developer.orgBankruptMsgDttm,data.metro.id,data.metro.name,data.metro.line,data.metro.color,data.metro.time,data.metro.isWalk,data.metro.colors
60,0,41620,472265,7040,АЛЬФА,АЛЬФА,АЛЬФА,0,14,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5178,0,27397,91029,6336,СЗ МЕГАСТРОЙ,СЗ МЕГАСТРОЙ,СПЕЦИАЛИЗИРОВАННЫЙ ЗАСТРОЙЩИК МЕГАСТРОЙ,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9835,0,38296,380276,10395,СПЕЦЗАСТРОЙЩИК ЮИТ ГРИН,СПЕЦЗАСТРОЙЩИК ЮИТ ГРИН,СПЕЦИАЛИЗИРОВАННЫЙ ЗАСТРОЙЩИК ЮИТ ГРИН,0,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8879,0,40339,461353,11098,СЗ-КОНСТРУКТОР,СЗ-КОНСТРУКТОР,СПЕЦИАЛИЗИРОВАННЫЙ ЗАСТРОЙЩИК-КОНСТРУКТОР,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3603,0,41054,449238,11823,СЗ ДОМИР,СЗ ДОМИР,СПЕЦИАЛИЗИРОВАННЫЙ ЗАСТРОЙЩИК ДОМА ИРКУТСКА,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


По части колонок у нас есть пропуски. Необходимо будет их обработать заполнив пропуски

### **Сохраняем полученный датафрейм в различные форматы**

### *Сохраняем в Excel*

In [5]:
df.to_excel('df_dmrf.xlsx')

### *Сохраняем в Pickle*

In [6]:
df.to_pickle('df_dmrf.pkl')

### *Сохраняем в БД*

In [7]:
con = sqlite3.connect('ds_repo.db')

In [8]:
df = df.applymap(str)
df.to_sql('df_dmrf', con, if_exists = 'replace')

In [9]:
con.commit()
con.close()